In [1]:
import sys
import re
import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
  
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LogisticRegression
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', con = engine)
X = df.iloc[:, 1]
Y = df.iloc[:, 4:]
print(X.head(2))
Y.head(2)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
Name: message, dtype: object


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text, language = 'english', lem = True):
    ## normalized
    text_norm = re.sub(r'[^a-zA-Z0-9]'," ", text.lower())
    ## tokenized
    word_text = word_tokenize(text_norm)
    ## skip stop word
    stop = stopwords.words(language)
    words = [w for w in word_text if w not in stop]
    if lem:
        return [WordNetLemmatizer().lemmatize(w) for w in words]
    else:
        return [PorterStemmer().stem(w) for w in words]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [8]:
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [4]:
def display_results(y_test, y_pred, i):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred)
    accuracy = sum(y_pred == y_test)/len(y_pred)
    
    Labels =  Y.columns[i]
    Confusion_Matrix =  confusion_mat
    accuracy = accuracy
    return labels, Confusion_Matrix, accuracy



In [5]:
def model_accuracy(y_pred, y_test, Y):
    accuracy_lst = []
    y_pred_df = pd.DataFrame(data = y_pred, columns = Y.columns, index = y_test.index)
    for i in range(y_pred_df.shape[1]):
        labels, Confusion_Matrix, accuracy = display_results(y_test.iloc[:,i], y_pred_df.iloc[:,i], i)
        accuracy_lst.append(accuracy)
    print('Mean accuracy of Model : {}'.format(np.mean(accuracy_lst)))
    return accuracy_lst

### Message variable model ( CountVectorizer/ TfidfVectorizer/ RandomForestClassifier )

In [11]:
model_accuracy(y_pred, y_test, Y)

Mean accuracy of Model : 0.9452115756281153


### 6. Improve your model
Use grid search to find better parameters. 

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [7]:
def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

    parameters = {
        'clf__estimator__n_estimators':  [10,20],
        'clf__estimator__min_samples_split': [2, 3]
    }

    cv = GridSearchCV(estimator = pipeline, param_grid=parameters)

    return cv

In [8]:
def main():
    model = build_model()
    print('training...')
    model.fit(X_train, y_train)
    print(model.best_params_)
    y_pred = model.predict(X_test)
    accuracy_lst = model_accuracy(y_pred, y_test, Y)
    filename = 'finalized_model.pkl'
    pickle.dump(model, open(filename, 'wb'))
    return y_pred, accuracy_lst

In [9]:
y_pred, accuracy_lst = main()

training...
{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 20}
Mean accuracy of Model : 0.9475383989421222


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### Use both message and genre as X variables

In [69]:
X = df.iloc[:, [1,3]]
Y = df.iloc[:, 4:]

In [44]:
def first_column(X):
    return X.iloc[:, 0]

def second_column(X):
    return X.iloc[:, 1]

# pipeline to get all tfidf and word count for first column
pipeline_one = Pipeline([
    ('column_selection', FunctionTransformer(first_column, validate=False)),
    ('feature-extractors', FeatureUnion([('tfidf', TfidfVectorizer()),
                                        ('counts', CountVectorizer())

    ]))
])

# Then a second pipeline to do the same for the second column
pipeline_two = Pipeline([
    ('column_selection', FunctionTransformer(second_column, validate=False)),
    ('feature-extractors', FeatureUnion([('tfidf', TfidfVectorizer()),
                                        ('counts', CountVectorizer())

    ]))
])


# Then you would again feature union these pipelines 
# to get different feature selection for each column

pipeline = Pipeline([
    ('features', FeatureUnion([('first-column-features', pipeline_one),
                                  ('second-column-feature', pipeline_two)])),
   ('clf', MultiOutputClassifier(RandomForestClassifier()))])


In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
model_accuracy(y_pred, y_test, Y)

Mean accuracy of Model : 0.9443172956294714


In [40]:
def build_model():
    pipeline_one = Pipeline([
    ('column_selection', FunctionTransformer(first_column, validate=False)),
    ('feature-extractors', FeatureUnion([('tfidf', TfidfVectorizer()),
                                        ('counts', CountVectorizer())

    ]))
])
    pipeline_two = Pipeline([
    ('column_selection', FunctionTransformer(second_column, validate=False)),
    ('feature-extractors', FeatureUnion([('tfidf', TfidfVectorizer()),
                                        ('counts', CountVectorizer())

    ]))
]) 
    pipeline = Pipeline([
    ('features', FeatureUnion([('first-column-features', pipeline_one),
                                  ('second-column-feature', pipeline_two)])),
   ('clf', MultiOutputClassifier(RandomForestClassifier()))])

    
    parameters = {
        'clf__estimator__n_estimators':  [10,20],
        'clf__estimator__min_samples_split': [2, 3]
    }

    cv = GridSearchCV(estimator = pipeline, param_grid=parameters)

    return cv

### Use GridSearch for Two Variable model ( TfidfVectorizer/ CountVectorizer/ RandomForestClassifier )

In [41]:
y_pred, accuracy_lst = main()

training...
{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 20}
Mean accuracy of Model : 0.945796460176991


### Add other features besides the TF-IDF

In [52]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        try:
            sentence_list = nltk.sent_tokenize(text)
            if sentence_list == []:
                return False
            for sentence in sentence_list:
                try:
                    pos_tags = nltk.pos_tag(tokenize(sentence))
                    first_word, first_tag = pos_tags[0]
                    if first_tag in ['VB', 'VBP']:
                        return True
                    else:
                        return False
                except:
                    return False
        except:
            return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


In [55]:
def first_column(X):
    return X.iloc[:, 0]

def second_column(X):
    return X.iloc[:, 1]

# pipeline to get all tfidf and word count for first column

pipeline_one = Pipeline([
    ('column_selection', FunctionTransformer(first_column, validate=False)),
    ('features', FeatureUnion([
                    ('text',Pipeline([
                                 ('vect', CountVectorizer(tokenizer=tokenize)),
                                ('tfidf', TfidfTransformer())
                             ])), 
                    ('starting_verb', StartingVerbExtractor())]))
    ])



# Then a second pipeline to do the same for the second column
pipeline_twe = Pipeline([
    ('column_selection', FunctionTransformer(second_column, validate=False)),
    ('features', FeatureUnion([
                    ('text',Pipeline([
                                 ('vect', CountVectorizer(tokenizer=tokenize)),
                                ('tfidf', TfidfTransformer())
                             ])), 
                    ('starting_verb', StartingVerbExtractor())]))
    ])


# Then you would again feature union these pipelines 
# to get different feature selection for each column

pipeline = Pipeline([
    ('features', FeatureUnion([('first-column-features', pipeline_one),
                                  ('second-column-feature', pipeline_two)])),
   ('clf', MultiOutputClassifier(RandomForestClassifier()))])

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)
# Your dataframe has your target as the first column, so make sure to drop first


In [56]:
model_accuracy(y_pred, y_test, Y)

Mean accuracy of Model : 0.9457498389448344


In [58]:
def build_model():
    # pipeline to get all tfidf and word count for first column

    pipeline_one = Pipeline([
    ('column_selection', FunctionTransformer(first_column, validate=False)),
    ('features', FeatureUnion([
                    ('text',Pipeline([
                                 ('vect', CountVectorizer(tokenizer=tokenize)),
                                ('tfidf', TfidfTransformer())
                             ])), 
                    ('starting_verb', StartingVerbExtractor())]))
    ])



# Then a second pipeline to do the same for the second column
    pipeline_twe = Pipeline([
    ('column_selection', FunctionTransformer(second_column, validate=False)),
    ('features', FeatureUnion([
                    ('text',Pipeline([
                                 ('vect', CountVectorizer(tokenizer=tokenize)),
                                ('tfidf', TfidfTransformer())
                             ])), 
                    ('starting_verb', StartingVerbExtractor())]))
    ])


# Then you would again feature union these pipelines 
# to get different feature selection for each column

    pipeline = Pipeline([
    ('features', FeatureUnion([('first-column-features', pipeline_one),
                                  ('second-column-feature', pipeline_two)])),
   ('clf', MultiOutputClassifier(RandomForestClassifier()))])

    
    parameters = {
        'clf__estimator__n_estimators':  [10,20],
        'clf__estimator__min_samples_split': [2, 3]
    }

    cv = GridSearchCV(estimator = pipeline, param_grid=parameters)

    return cv

In [59]:
y_pred, accuracy_lst = main()

training...
{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 20}
Mean accuracy of Model : 0.9474324415963111


### Use AdaBoostClassifier

In [76]:
def first_column(X):
    return X.iloc[:, 0]

def second_column(X):
    return X.iloc[:, 1]

# pipeline to get all tfidf and word count for first column

pipeline_one = Pipeline([
    ('column_selection', FunctionTransformer(first_column, validate=False)),
    ('features', FeatureUnion([
                    ('text',Pipeline([
                                 ('vect', CountVectorizer(tokenizer=tokenize)),
                                ('tfidf', TfidfTransformer())
                             ])), 
                    ('starting_verb', StartingVerbExtractor())]))
    ])



# Then a second pipeline to do the same for the second column
pipeline_twe = Pipeline([
    ('column_selection', FunctionTransformer(second_column, validate=False)),
    ('features', FeatureUnion([
                    ('text',Pipeline([
                                 ('vect', CountVectorizer(tokenizer=tokenize)),
                                ('tfidf', TfidfTransformer())
                             ])), 
                    ('starting_verb', StartingVerbExtractor())]))
    ])


# Then you would again feature union these pipelines 
# to get different feature selection for each column

pipeline = Pipeline([
    ('features', FeatureUnion([('first-column-features', pipeline_one),
                                  ('second-column-feature', pipeline_two)])),
   ('clf', MultiOutputClassifier(AdaBoostClassifier()))])

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)
# Your dataframe has your target as the first column, so make sure to drop first


In [77]:
model_accuracy(y_pred, y_test, Y)

Mean accuracy of Model : 0.9472798630183435


### Use GridSearch with AdaBoostClassifier

In [90]:
def build_model():
    # pipeline to get all tfidf and word count for first column

    pipeline_one = Pipeline([
    ('column_selection', FunctionTransformer(first_column, validate=False)),
    ('features', FeatureUnion([
                    ('text',Pipeline([
                                 ('vect', CountVectorizer(tokenizer=tokenize)),
                                ('tfidf', TfidfTransformer())
                             ])), 
                    ('starting_verb', StartingVerbExtractor())]))
    ])



# Then a second pipeline to do the same for the second column
    pipeline_twe = Pipeline([
    ('column_selection', FunctionTransformer(second_column, validate=False)),
    ('features', FeatureUnion([
                    ('text',Pipeline([
                                 ('vect', CountVectorizer(tokenizer=tokenize)),
                                ('tfidf', TfidfTransformer())
                             ])), 
                    ('starting_verb', StartingVerbExtractor())]))
    ])


# Then you would again feature union these pipelines 
# to get different feature selection for each column

    pipeline = Pipeline([
    ('features', FeatureUnion([('first-column-features', pipeline_one),
                                  ('second-column-feature', pipeline_two)])),
   ('clf', MultiOutputClassifier(AdaBoostClassifier()))])
    
    
    parameters = {
        'clf__estimator__n_estimators':  [10,20],
        'clf__estimator__learning_rate': [0.005, 0.01, 0.01]
    }

    cv = GridSearchCV(estimator = pipeline, param_grid=parameters)

    return cv

In [92]:
y_pred, accuracy_lst = main()

dict_keys(['memory', 'steps', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__first-column-features', 'features__second-column-feature', 'features__first-column-features__memory', 'features__first-column-features__steps', 'features__first-column-features__column_selection', 'features__first-column-features__features', 'features__first-column-features__column_selection__accept_sparse', 'features__first-column-features__column_selection__func', 'features__first-column-features__column_selection__inv_kw_args', 'features__first-column-features__column_selection__inverse_func', 'features__first-column-features__column_selection__kw_args', 'features__first-column-features__column_selection__pass_y', 'features__first-column-features__column_selection__validate', 'features__first-column-features__features__n_jobs', 'features__first-column-features__features__transformer_list', 'features__first-column-features__features__transform

### 9. Export your model as a pickle file

In [ ]:
filename = 'finalized_model.pkl'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.